# Generate final result according to bert prediction

In [11]:
import pandas as pd 
import json 

data_to_predict = pd.read_csv('bert_dev_shuffle_top5_index.tsv', sep='\t')
predict_results = pd.read_csv('1epoch_dev_top5_results.tsv', sep='\t')
with open('devset.json', 'r') as f: 
    origin_data = json.load(f)

data_to_predict['true'] = predict_results['true']


In [12]:
import ast

result_dict = {}

for index, row in data_to_predict.iterrows():
    key = str(row[0])
    claim = row[1]
    evi_index = ast.literal_eval(row[2])
    probability = row[5]
    if probability > 0.45: 
        if key in result_dict.keys():
            result_dict[key]['evidence'].append(evi_index)
        else:
            result_dict.update({
                key:{
                    "claim": claim,
                    "label": "NOT ENOUGH INFO",
                    "evidence": [evi_index]
                }
            })

In [13]:
len(result_dict)

2923

In [14]:
for key in origin_data.keys():
    item = origin_data[key]
    if key in result_dict.keys():
        continue
    else:
        result_dict.update({
            key:{
                "claim": item['claim'],
                "label": item['label'],
                "evidence": []
            }
        })
len(result_dict)


5001

In [15]:
with open('final_result.json', 'w') as o:
    json.dump(result_dict, o, indent=4)